# **THE MNIST DATABASE of handwritten digits**

## **1) Importação & preprocessamento dos dados**

In [27]:
import urllib.request
import gzip
import pickle
import os
import numpy as np
from PIL import Image

def _download(file_name):
    file_path = dataset_dir + "/" + file_name

    if os.path.exists(file_path):
        return

    print("Downloading " + file_name + " ... ")
    urllib.request.urlretrieve(url_base + file_name, file_path)
    print("Done")

def download_mnist():
    for v in key_file.values():
       _download(v)

def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")

    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name

    print("Converting " + file_name + " to NumPy Array ...")
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, img_size)
    print("Done")

    return data

def _convert_numpy():
    dataset = {}
    dataset['train_img'] =  _load_img(key_file['train_img'])
    dataset['train_label'] = _load_label(key_file['train_label'])
    dataset['test_img'] = _load_img(key_file['test_img'])
    dataset['test_label'] = _load_label(key_file['test_label'])

    return dataset

def init_mnist():
    download_mnist()
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("Done")

def _change_ont_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1

    return T

def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """
    Parameters
    ----------
    normalize : Normalize the pixel values
    flatten : Flatten the images as one array
    one_hot_label : Encode the labels as a one-hot array

    Returns
    -------
    (Trainig Image, Training Label), (Test Image, Test Label)
    """
    if not os.path.exists(save_file):
        init_mnist()

    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)

    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0

    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    if one_hot_label:
        dataset['train_label'] = _change_ont_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_ont_hot_label(dataset['test_label'])

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label'])

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()


# Load the MNIST dataset
url_base = 'http://yann.lecun.com/exdb/mnist/'
key_file = {
    'train_img':'train-images-idx3-ubyte.gz',
    'train_label':'train-labels-idx1-ubyte.gz',
    'test_img':'t10k-images-idx3-ubyte.gz',
    'test_label':'t10k-labels-idx1-ubyte.gz'
}

dataset_dir = "C:/Users/Igor/Desktop/TÓPICOS 2/Bancos"
save_file = dataset_dir + "/mnist.pkl"

train_num = 60000
test_num = 10000
img_dim = (1, 28, 28)
img_size = 784


### Cada uma das 60000 imagens para treinamento, ou das 10000 imagens para teste, é uma matriz 28x28 em que cada entrada varia de 0 até 255

### Cada uma das 784 entradas, elemento (i,j) da matriz, representa a intensidade do branco no pixel (i,j)

In [48]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=True)

# Show the sample image
img = x_train[0]
label = t_train[0]

print("Cada imagem foi achatada em um vetor coluna assim:",img.shape)
img = img.reshape(28, 28)
print("Coloca-se a imagem, de volta para sua forma original, em uma matriz 28x28, em que cada entrada dessa matrix é a intesidade do branco no pixel (i,j):",img.shape)


Cada imagem foi achatada em um vetor coluna assim: (784,)
Coloca-se a imagem, de volta para sua forma original, em uma matriz 28x28, em que cada entrada dessa matrix é a intesidade do branco no pixel (i,j): (28, 28)


In [46]:
img_show(img)

print("A classificação correta para a primeira imagem de treinamento é:",label)

A classificação correta para a primeira imagem de treinamento é: 5


### Faz-se esse achatamento das matrizes 28x28 em um vetor $\vec{x}$ coluna 784x1 para que:

### i) $\vec{x}$ seja uma observação no $ \mathbb{R}^{784}$

### Então seria possível classificar $\vec{x}$, em particular, a partir de SVM's.

### ii) $\vec{x}$ pode ser classificado como um número inteiro de 0 até 9


## **2) EDA**